In [5]:
import re
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf

In [8]:

# Specify relative path to the file
file_path = "../IAPS/banco_completo_modificado.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')


In [10]:

df['cir_proc_tto_min_rec'] = df['cir_proc_tto_min_rec'].replace('CIRURGIAS MULTIPLAS E SEQUENCIAIS', 'multiplas_cir_seq')

# Convert categorical variables to one-hot encoded columns
categorical_cols =['sexo', 'racacor', 'estadio', 'tipo_proc_min', 'proc_max', 
    'desc_f_org_tto_min', 'tipo_tto_min', 'perfil', 'estadio_rec','cidpri_min', 'cid_tto_min',
    'f_org_tto_min_rec', 'perfil_rec','cir_proc_tto_min_rec', 'cir_proc_tto_min_rec2',
    'cump_lei60dias', 'macro_tto', 'mun_pcn', 'regiao_pcn', 'macro_pcn', 'estabel_saude', 'cidade_estabel',
    'estado_estabel', 'regiao_estabel', 'macro_estabel', 'regiao_bras_estabel', 'finaliquimio',
    'finaliradio', 'time_diag']  
df_encoded = pd.get_dummies(df, columns=categorical_cols)

# Convert numeric variables to the correct data type
numeric_cols = ['qt_apac', 'qt_aih', 'idade', 'n_comorb', 'tempo_diag_tto_min', 'tempo_internacao', 'tempo_coorte']  
df_encoded[numeric_cols] = df_encoded[numeric_cols].astype(float)

# Apply Z-score normalization to numeric columns
scaler = StandardScaler()
df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])

# Prepare the data
X = df_encoded.drop('obito', axis=1)  # Features
y = df_encoded['obito']  # Target variable


In [ ]:
df_encoded

In [19]:
LISTA=[[0 for i in range(3)] for i in range(100)]

In [ ]:
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    
    # Create XGBoost classifier
    model = xgb.XGBClassifier(use_label_encoder=False, objective='binary:logistic', eval_metric='logloss')

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    LISTA[i][0] = accuracy_score(y_test, y_pred)


    
    # Train the logistic regression model
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    LISTA[i][1] = accuracy_score(y_test, y_pred)


    
    
    
    # New Neural Network Model using Keras
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile with an optimizer that has a learning rate schedule
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early Stopping to monitor the validation loss and stop when it starts increasing
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model with validation data and early stopping
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

    # Make predictions on the test set
    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    # Evaluate the model
    LISTA[i][2] = accuracy_score(y_test, y_pred)




2023-08-13 00:50:30.404195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
120/120 [==============================] - 1s 3ms/step - loss: 1.1569 - accuracy: 0.6834 - val_loss: 0.7836 - val_accuracy: 0.6764
Epoch 2/50
120/120 [==============================] - 0s 2ms/step - loss: 0.7039 - accuracy: 0.7137 - val_loss: 0.6527 - val_accuracy: 0.7328
Epoch 3/50
120/120 [==============================] - 0s 2ms/step - loss: 0.6148 - accuracy: 0.7413 - val_loss: 0.6141 - val_accuracy: 0.7432
Epoch 4/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5897 - accuracy: 0.7465 - val_loss: 0.5991 - val_accuracy: 0.7463
Epoch 5/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5783 - accuracy: 0.7515 - val_loss: 0.5952 - val_accuracy: 0.7328
Epoch 6/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5666 - accuracy: 0.7510 - val_loss: 0.5824 - val_accuracy: 0.7505
Epoch 7/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5591 - accuracy: 0.7578 - val_loss: 0.5752 - val_accuracy: 0.7557
Epoch 

120/120 [==============================] - 0s 2ms/step - loss: 0.5673 - accuracy: 0.7518 - val_loss: 0.5338 - val_accuracy: 0.7829
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5728 - accuracy: 0.7552 - val_loss: 0.5393 - val_accuracy: 0.7714
Epoch 10/50
30/30 [==============================] - 0s 785us/step
Epoch 1/50
120/120 [==============================] - 1s 3ms/step - loss: 1.1740 - accuracy: 0.6719 - val_loss: 0.7704 - val_accuracy: 0.7015
Epoch 2/50
120/120 [==============================] - 0s 2ms/step - loss: 0.7120 - accuracy: 0.7285 - val_loss: 0.6250 - val_accuracy: 0.7516
Epoch 3/50
120/120 [==============================] - 0s 2ms/step - loss: 0.6327 - accuracy: 0.7418 - val_loss: 0.5865 - val_accuracy: 0.7610
Epoch 4/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5998 - accuracy: 0.7437 - val_loss: 0.5660 - val_accuracy: 0.7610
Epoch 5/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5782 - accuracy:

120/120 [==============================] - 0s 2ms/step - loss: 0.5550 - accuracy: 0.7578 - val_loss: 0.5537 - val_accuracy: 0.7630
Epoch 15/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5488 - accuracy: 0.7604 - val_loss: 0.5532 - val_accuracy: 0.7610
Epoch 16/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5472 - accuracy: 0.7648 - val_loss: 0.5539 - val_accuracy: 0.7599
Epoch 17/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5461 - accuracy: 0.7674 - val_loss: 0.5527 - val_accuracy: 0.7589
Epoch 18/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5474 - accuracy: 0.7693 - val_loss: 0.5496 - val_accuracy: 0.7620
Epoch 19/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5513 - accuracy: 0.7669 - val_loss: 0.5520 - val_accuracy: 0.7672
Epoch 20/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5520 - accuracy: 0.7651 - val_loss: 0.5512 - val_accuracy: 0.7850
Epoch 21/50

120/120 [==============================] - 0s 2ms/step - loss: 0.5544 - accuracy: 0.7646 - val_loss: 0.5664 - val_accuracy: 0.7547
Epoch 14/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5496 - accuracy: 0.7638 - val_loss: 0.5623 - val_accuracy: 0.7557
Epoch 15/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5436 - accuracy: 0.7729 - val_loss: 0.5644 - val_accuracy: 0.7453
Epoch 16/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5525 - accuracy: 0.7677 - val_loss: 0.5672 - val_accuracy: 0.7505
Epoch 17/50
30/30 [==============================] - 0s 757us/step
Epoch 1/50
120/120 [==============================] - 1s 3ms/step - loss: 1.1740 - accuracy: 0.6682 - val_loss: 0.7557 - val_accuracy: 0.7244
Epoch 2/50
120/120 [==============================] - 0s 2ms/step - loss: 0.6990 - accuracy: 0.7270 - val_loss: 0.6198 - val_accuracy: 0.7610
Epoch 3/50
120/120 [==============================] - 0s 2ms/step - loss: 0.6265 - accura

...

120/120 [==============================] - 0s 2ms/step - loss: 0.5879 - accuracy: 0.7478 - val_loss: 0.5507 - val_accuracy: 0.7672
Epoch 6/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5780 - accuracy: 0.7450 - val_loss: 0.5403 - val_accuracy: 0.7693
Epoch 7/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5717 - accuracy: 0.7512 - val_loss: 0.5420 - val_accuracy: 0.7735
Epoch 8/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5641 - accuracy: 0.7541 - val_loss: 0.5382 - val_accuracy: 0.7735
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5641 - accuracy: 0.7510 - val_loss: 0.5378 - val_accuracy: 0.7724
Epoch 10/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5647 - accuracy: 0.7536 - val_loss: 0.5432 - val_accuracy: 0.7704
Epoch 11/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5585 - accuracy: 0.7604 - val_loss: 0.5318 - val_accuracy: 0.7777
Epoch 12/50
120

120/120 [==============================] - 0s 2ms/step - loss: 0.5816 - accuracy: 0.7515 - val_loss: 0.5555 - val_accuracy: 0.7495
Epoch 7/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5704 - accuracy: 0.7528 - val_loss: 0.5456 - val_accuracy: 0.7557
Epoch 8/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5642 - accuracy: 0.7463 - val_loss: 0.5436 - val_accuracy: 0.7630
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5597 - accuracy: 0.7585 - val_loss: 0.5430 - val_accuracy: 0.7620
Epoch 10/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5616 - accuracy: 0.7497 - val_loss: 0.5423 - val_accuracy: 0.7756
Epoch 11/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5604 - accuracy: 0.7518 - val_loss: 0.5512 - val_accuracy: 0.7693
Epoch 12/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5642 - accuracy: 0.7533 - val_loss: 0.5420 - val_accuracy: 0.7777
Epoch 13/50
12

120/120 [==============================] - 0s 2ms/step - loss: 0.6356 - accuracy: 0.7439 - val_loss: 0.5952 - val_accuracy: 0.7610
Epoch 4/50
120/120 [==============================] - 0s 2ms/step - loss: 0.6009 - accuracy: 0.7505 - val_loss: 0.5728 - val_accuracy: 0.7589
Epoch 5/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5843 - accuracy: 0.7541 - val_loss: 0.5564 - val_accuracy: 0.7537
Epoch 6/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5757 - accuracy: 0.7512 - val_loss: 0.5581 - val_accuracy: 0.7568
Epoch 7/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5643 - accuracy: 0.7567 - val_loss: 0.5511 - val_accuracy: 0.7599
Epoch 8/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5614 - accuracy: 0.7612 - val_loss: 0.5579 - val_accuracy: 0.7630
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5626 - accuracy: 0.7609 - val_loss: 0.5521 - val_accuracy: 0.7537
Epoch 10/50
120/1

120/120 [==============================] - 0s 2ms/step - loss: 0.5552 - accuracy: 0.7572 - val_loss: 0.5473 - val_accuracy: 0.7537
Epoch 13/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5585 - accuracy: 0.7617 - val_loss: 0.5500 - val_accuracy: 0.7599
Epoch 14/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5552 - accuracy: 0.7606 - val_loss: 0.5510 - val_accuracy: 0.7589
Epoch 15/50
30/30 [==============================] - 0s 765us/step
Epoch 1/50
120/120 [==============================] - 1s 3ms/step - loss: 1.1885 - accuracy: 0.6896 - val_loss: 0.8345 - val_accuracy: 0.6775
Epoch 2/50
120/120 [==============================] - 0s 2ms/step - loss: 0.7169 - accuracy: 0.7301 - val_loss: 0.6912 - val_accuracy: 0.7098
Epoch 3/50
120/120 [==============================] - 0s 2ms/step - loss: 0.6207 - accuracy: 0.7460 - val_loss: 0.6291 - val_accuracy: 0.7317
Epoch 4/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5883 - accurac

120/120 [==============================] - 0s 2ms/step - loss: 0.5730 - accuracy: 0.7512 - val_loss: 0.5337 - val_accuracy: 0.7683
Epoch 8/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5702 - accuracy: 0.7531 - val_loss: 0.5314 - val_accuracy: 0.7610
Epoch 9/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5622 - accuracy: 0.7531 - val_loss: 0.5259 - val_accuracy: 0.7724
Epoch 10/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5659 - accuracy: 0.7544 - val_loss: 0.5296 - val_accuracy: 0.7724
Epoch 11/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5597 - accuracy: 0.7567 - val_loss: 0.5218 - val_accuracy: 0.7787
Epoch 12/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5580 - accuracy: 0.7612 - val_loss: 0.5252 - val_accuracy: 0.7766
Epoch 13/50
120/120 [==============================] - 0s 2ms/step - loss: 0.5656 - accuracy: 0.7531 - val_loss: 0.5262 - val_accuracy: 0.7829
Epoch 14/50
3

In [ ]:
LIST = LISTA[0:100]

In [ ]:
import matplotlib.pyplot as plt

# Assuming LIST is a list of lists, where each inner list has 3 accuracy values
# for XGBoost, Logistic Regression, and the Neural Network respectively.

# Convert LIST into a numpy array for easier indexing
import numpy as np
LIST = np.array(LIST)

# X values (trial numbers)
x_values = range(1, 101)  # Assuming you've done this for 30 trials

# Plotting
plt.figure(figsize=(12, 6))

# Plotting each model's results
plt.plot(x_values, LIST[:, 0], marker='o', label='XGBoost', color='blue')
plt.plot(x_values, LIST[:, 1], marker='o', label='Logistic Regression', color='green')
plt.plot(x_values, LIST[:, 2], marker='o', label='Neural Network', color='red')

# Drawing horizontal lines for means
plt.axhline(y=np.mean(LIST[:, 0]), color='blue', linestyle='--', label='XGBoost Mean')
plt.axhline(y=np.mean(LIST[:, 1]), color='green', linestyle='--', label='Logistic Regression Mean')
plt.axhline(y=np.mean(LIST[:, 2]), color='red', linestyle='--', label='Neural Network Mean')

# Setting labels, title, and legend
plt.xlabel('#Trial Number')
plt.ylabel('Accuracy')
plt.title('Comparison of Model Accuracies Across Different Trials')
plt.legend()

# Display the plot
plt.grid(True)
plt.tight_layout()
plt.show()
